In [35]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, ParameterGrid, KFold
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm, trange 

In [36]:
df = pd.read_csv("house_corrected_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2254 entries, 0 to 2253
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Количество комнат  2254 non-null   int64  
 1   Площадь (м²)       2254 non-null   float64
 2   Адрес              2254 non-null   object 
 3   Цена ($)           2254 non-null   int64  
 4   Цена (сом)         2254 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 88.2+ KB


In [37]:
features = ["Количество комнат", "Площадь (м²)"]
target = "Цена ($)"  # Или "Цена (сом)", если требуется

X = df[features]
y = df[target]

# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определяем сетку гиперпараметров для поиска
param_grid = {
    'iterations': [1300, 1500, 1700],
    'learning_rate': [0.07, 0.08],
    'depth': [8]
}
grid = list(ParameterGrid(param_grid))
kf = KFold(n_splits=3, shuffle=True, random_state=42)

best_score = float('inf')
best_params = None

print("Начало GridSearch:")

# Перебор всех комбинаций гиперпараметров с использованием tqdm для отображения прогресса
for params in tqdm(grid, desc="Параметрические комбинации", leave=True):
    cv_scores = []
    # 3-кратная кросс-валидация
    for train_index, val_index in kf.split(X_train):
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]
        # Создаем модель с текущими параметрами
        model = CatBoostRegressor(
            allow_writing_files=False,
            random_seed=42,
            task_type='GPU',          # Используем GPU для ускорения обучения
            eval_metric='RMSE',
            early_stopping_rounds=50,
            verbose=0,
            **params
        )
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val))
        y_pred_val = model.predict(X_val)
        score = mean_absolute_error(y_val, y_pred_val)
        cv_scores.append(score)
    mean_score = np.mean(cv_scores)
    if mean_score < best_score:
        best_score = mean_score
        best_params = params

print("Лучшие параметры:", best_params)
print("Лучший CV MAE:", best_score)

# Обучаем итоговую модель на полном обучающем наборе с найденными параметрами
best_model = CatBoostRegressor(
    allow_writing_files=False,
    random_seed=42,
    task_type='GPU',
    eval_metric='RMSE',
    early_stopping_rounds=50,
    verbose=100,  # Вывод прогресса обучения итоговой модели
    **best_params
)
best_model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Оценка модели на тестовой выборке
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Test MAE:", mae)
print("Test R2:", r2)

# Функция для предсказания цены квартиры по вводимым параметрам (без столбца "Адрес")
def predict(num_rooms, area):
    input_data = pd.DataFrame({
        "Количество комнат": [num_rooms],
        "Площадь (м²)": [area],
    })
    return best_model.predict(input_data)[0]

# Пример использования функции
predicted_price = predict(6, 300)
print("Предсказанная цена дома:", predicted_price)

Начало GridSearch:


Параметрические комбинации: 100%|██████████| 6/6 [00:19<00:00,  3.19s/it]


Лучшие параметры: {'depth': 8, 'iterations': 1300, 'learning_rate': 0.07}
Лучший CV MAE: 105225.73118361922
0:	learn: 608466.4445482	test: 205823.9604638	best: 205823.9604638 (0)	total: 12.1ms	remaining: 15.7s
bestTest = 168243.8565
bestIteration = 11
Shrink model to first 12 iterations.
Test MAE: 106921.88550430602
Test R2: 0.38675678214734877
Предсказанная цена дома: 333864.23974609375


In [38]:
# Сохраняем модель в файл
best_model.save_model("house_catboost_model.cbm")